MS Studio

When entering a new business, Microsoft has one tradition. Strive for excellence. Our goal with Microsoft Studio is to perpetuate this tradition. 

Every year, thousands of new movies are released. Some are acclaimed by the critics; others are box office success. Our goal is to release movies that are both, critic and box office success. 
First, less take a look on this 2x2 matrice of all the movies that were released during the last 10 years.

In [ ]:
#Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
!pip install mysql-connector-python

In [ ]:
#Connecting SQL

import mysql.connector
con = mysql.connector.connect(user = 'hbs-student', password = 'super-secure-password?', host = 'hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com', database = 'movies')
cur = con.cursor()

In [ ]:
#Creating df_all from IMDB ratings

cur.execute("""SELECT *
                FROM movies.imdb_title_ratings
                JOIN movies.imdb_title_basics USING(tconst)
                WHERE imdb_title_basics.start_year >= 2010
                AND imdb_title_ratings.numvotes > 50;""")



df_all = pd.DataFrame(cur.fetchall())
df_all.columns = [x[0] for x in cur.description]

# Remove special characters
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df_all['primary_title'] = df_all['primary_title'].str.replace(char, ' ')
    
#put Title in low caps
df_all['primary_title'] =  df_all['primary_title'].map(lambda x: x.lower())

df_all.head(2)

In [ ]:
#Creating df_roi from tn_movie_budgets

df_roi = pd.read_sql_query('select * from tn_movie_budgets', con)
df_roi ["release_date"] = pd.to_datetime(df_roi ["release_date"]) 
df_roi.head()

#Cleaning those 3 columns
df_roi ["production_budget"] = df_roi ["production_budget"].map(lambda x: x.lstrip('$')).str.replace(',','')
df_roi ["domestic_gross"] = df_roi ["domestic_gross"].map(lambda x: x.strip('$,')).str.replace(',','')
df_roi ["worldwide_gross"] = df_roi ["worldwide_gross"].map(lambda x: x.strip('$,')).str.replace(',','')
df_roi = df_roi.astype({"production_budget":'int64', "domestic_gross":'int64',"worldwide_gross":'int64'}) 

#New column roi metrics that show you 
df_roi ["roi"] = (df_roi ["worldwide_gross"])/df_roi ["production_budget"]
df_roi = df_roi.sort_values("roi",ascending = 0)

# Selecting movies after 2000
df_roi['year'] = df_roi['release_date'].apply(lambda x: x.year)
df_roi = df_roi.loc [df_roi['year'] > 2010]

#Rename column
df_roi = df_roi.rename(columns={'movie': 'primary_title'})

# Remove special characters
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df_roi['primary_title'] = df_roi['primary_title'].str.replace(char, ' ')
    
#put Title in low caps
df_roi['primary_title'] =  df_roi['primary_title'].map(lambda x: x.lower())

#Rename column
df_roi = df_roi.rename(columns={'movie': 'primary_title'})

df_roi.head()

In [ ]:
#Left merge between df_roi and df_all
left_join_df = df_roi.merge(df_all, on='primary_title', how='left' )

#Drop if ROI or Rating is NA
left_join_df = left_join_df.dropna(subset = ["averagerating", "roi"])

In [ ]:
plt.scatter(left_join_df['roi'] , left_join_df['averagerating'], marker="o", s=1)
plt.xlim(0, 20)
plt.ylim(0, 10)

plt.hlines(y=6.5, xmin=5, xmax=20, color='red')
plt.vlines(x=5, ymin=6.5, ymax=10, color='red')


plt.title("Scatter Plot of the ROI and average ratings of the movies released since 2010")
plt.xlabel('ROI')
plt.ylabel('Rating') 

We are using 2 metrics to evaluate the movies:
- the average rating on IMDB, only for movies with more than 50 ratings
- the Return on Investment (ROI), which is the wordlwide revenue generated by a movie divided by the movie's budget

We will focus our presentation on movies that have a ROI over 5 and an average rating > 6.5

In [ ]:
#Importing SQL data (IMDB Ratings)

cur.execute("""SELECT *
                FROM movies.imdb_title_ratings
                JOIN movies.imdb_title_basics USING (tconst)
                JOIN movies.imdb_title_crew USING (tconst)
                LEFT JOIN movies.imdb_name_basics ON movies.imdb_name_basics.nconst = movies.imdb_title_crew.directors
                WHERE imdb_title_basics.start_year >= 2010
                AND imdb_title_ratings.averagerating >6.5
                AND imdb_title_ratings.numvotes > 50;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df



In [ ]:
#Spliting genres into columns

df[['Genres_1', 'Genres_2', 'Genres_3']] = df.genres.str.split(",", expand=True)
df.head(5)


In [ ]:
arr_genres = df.Genres_1.unique()
arr_genres

list_genres = arr_genres.tolist()
list_genres.remove('')

genres_count = []
for genre in list_genres:
    genres_count.append(df['genres'].apply(lambda x: genre in x).astype(np.uint8).sum())
genres_count 

#df['isAdventure'] = df['genres'].apply(lambda x: 'Adventure' in x).astype(np.uint8)
#total_adventure = df['isAdventure'].sum()

In [ ]:
#Connecting two lists to dictionary and transforming to DataFrame
#Sorting values by count (sum of all three genre types)

genres_cumulative = {'genres': list_genres, 'genres_count': genres_count}
genres_total = pd.DataFrame.from_dict(genres_cumulative)
genres_total.sort_values(by = 'genres_count', ascending = False)

In [ ]:
#Grouping by Genres1

genres1 = df.groupby('Genres_1').count().sort_values(by = 'tconst', ascending = False)
genres1 = genres1.tconst.reset_index()
genres1 = genres1.rename(columns={"tconst": "genres_count"})
genres1

#Grouping by Genres2
genres2 = df.groupby('Genres_2').count().sort_values(by = 'tconst', ascending = False)
genres2 = genres2.tconst.reset_index()
genres2 = genres2.rename(columns={"tconst": "genres_count"})
genres2

In [ ]:
#Renaming director column
df = df.rename(columns={"primary_name": "director_name"})
df.head(5)

In [ ]:
#Grouping top rated movies by directors

top_directors = df.groupby('director_name').count().sort_values(by = 'tconst', ascending = False)
top_directors = top_directors.tconst.reset_index()
top_directors = top_directors.rename(columns={"tconst": "movies_count"})
top_directors = top_directors.loc[top_directors['movies_count'] >= 6]
len(top_directors)

#There are 45 directors who produces 6+ movies with rating 6.5+ over last 10 years

In [ ]:
#Left join between df_roi et df
left_join_df_roi = df_roi.merge(df, on='primary_title', how='left' )
df_join = left_join_df_roi.dropna(subset = ["averagerating", "roi"])
